In [1]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel, AutoModel, pipeline
import torch
import re
import pandas as pd
clean = re.compile('<.*?>')


/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
pos_model_dict = torch.load('/home/ubuntu/ImdbPOS_model/pytorch_model.bin', map_location='cuda')
neg_model_dict = torch.load('/home/ubuntu/ImdbNeg_model/pytorch_model.bin', map_location='cuda') 
#ImdbPOS_model/pytorch_model.bin
#/home/ubuntu/ImdbPOS_model
#ImdbNeg_model

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [5]:
head_model_pos = GPT2LMHeadModel.from_pretrained('gpt2', state_dict=pos_model_dict, pad_token_id=tokenizer.eos_token_id).to(device)
head_model_neg = GPT2LMHeadModel.from_pretrained('gpt2', state_dict=neg_model_dict, pad_token_id=tokenizer.eos_token_id).to(device)
base_gpt2_head = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id).to(device)

In [6]:
text_neutral = "The park was"
prompts = ["I sat", "I thought", "School was", "The furnishings were", "Books like that are", "I find it", "Her grades were" ,"The smile was", "The outfit fit", "The boys habits are"]

In [7]:
text_ids = tokenizer.encode(text_neutral, return_tensors = 'pt').to("cuda")
text_ids

tensor([[ 464, 3952,  373]], device='cuda:0')

In [8]:
text_ids_list = []
for prompt in prompts:
    text_ids_list.append(tokenizer.encode(prompt, return_tensors = 'pt').to(device))

In [9]:
def generate_from_ids(model, text_ids_list):
    torch.manual_seed(0)
    model_generations = []
    for ids in text_ids_list:
        # generated = model.generate(ids, early_stopping=True, num_beams=20 ,no_repeat_ngram_size=2, max_length=100)
        generated = model.generate(ids, early_stopping=True, num_beams=5 ,no_repeat_ngram_size=2, do_sample=True, top_p=0.8, max_length=100)

        model_generations.append(generated)
    return model_generations

In [10]:
def remove_html_tags(text): # taken from medium https://medium.com/@jorlugaqui/how-to-strip-html-tags-from-a-string-in-python-7cb81a2bbf44
    """Remove html tags from a string"""
    return re.sub(clean, '', text)

In [11]:
pos_generated_text_samples = head_model_pos.generate(
    text_ids
)
pos_generated_text_samples

/home/ubuntu/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[  464,  3952,   373,   257,  1049,  1295,   284,   766,   262,  3807,
            13,   314,   373,   845, 12617,   351,   262,  3807,    13,   314]],
       device='cuda:0')

In [12]:
neg_generated_text_samples = head_model_neg.generate(
    text_ids
)
pos_generated_text_samples

tensor([[  464,  3952,   373,   257,  1049,  1295,   284,   766,   262,  3807,
            13,   314,   373,   845, 12617,   351,   262,  3807,    13,   314]],
       device='cuda:0')

In [13]:
base_generated_text_samples = base_gpt2_head.generate(
    text_ids
)
base_generated_text_samples

tensor([[ 464, 3952,  373, 4838,  329,  262, 1110,   11,  475,  262, 3952, 4706,
          531,  262, 3952,  373, 1280,  329, 1597,   13]], device='cuda:0')

In [14]:
# for i, beam in enumerate(base_generated_text_samples):
#     print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)}")

In [15]:
# for i, beam in enumerate(pos_generated_text_samples):
#     print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)}")

In [16]:
# for i, beam in enumerate(neg_generated_text_samples):
#     print(beam.shape)
#     print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)}")

In [17]:
# text_ids_list
# base_gpt2_head
# head_model_pos
# head_model_neg

In [18]:
sentiment_pipeline = pipeline(task = "sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [19]:
# generate from base model
base_generations = generate_from_ids(base_gpt2_head, text_ids_list)
decoded_base_list = []
base_model_labels = []
for i, completion in enumerate(base_generations):
    completion = completion.squeeze()
    decoded_base = tokenizer.decode(completion, skip_special_tokens=True)
    decoded_base = remove_html_tags(decoded_base).strip('\n').replace('\n','')
    decoded_base_list.append(decoded_base)
    base_model_labels.append(sentiment_pipeline(decoded_base)[0]['label'])
    print(f"{i}: {decoded_base}")

0: I sat down to write this article, and I wanted to share with you some of the things that I've learned from my time as a writer.1. I'm not afraid to ask for help. There are a lot of people out there who don't know what they're talking about. It's not uncommon to hear people say, "I'm scared to go to the doctor because I can't afford it, but I want to do something about it." I think that's a
1: I thought we were going to win," he said."It was just a matter of getting the ball out of the back of our net. We just had to get it back into the net and that's what we did."
2: School was founded in 1859, and it has since become one of the world's leading educational institutions.The school's mission is to provide students with the skills and knowledge needed to succeed in the 21st century. The school is dedicated to the pursuit of excellence in all areas of education, including the arts, science, technology, engineering and mathematics.
3: The furnishings were made in the same way as the fur

In [20]:
# generate from pos model
pos_generations = generate_from_ids(head_model_pos, text_ids_list)
decoded_pos_list = []
pos_model_labels = []
for i, completion in enumerate(pos_generations):
    completion = completion.squeeze()
    decoded_pos = tokenizer.decode(completion, skip_special_tokens=True)
    decoded_pos = remove_html_tags(decoded_pos).strip('\n').replace('\n','')
    decoded_pos_list.append(decoded_pos)
    pos_model_labels.append(sentiment_pipeline(decoded_pos)[0]['label'])
    print(f"{i}: {decoded_pos}")

0: I sat down to watch this movie and it was one of the best movies I have ever seen. I had no idea what I was going to get myself into when I saw it. It was a great movie, and I can't wait to see it again.This movie is a must-see for all of us.This is the first time I've seen a movie that doesn't have a lot to do with the plot, but it does have some interesting characters
1: I thought it was a good idea to make this film. I had no idea what I was going to do with it, but I really liked it.I was very impressed with the direction and direction of the film, and I would recommend it to anyone who is interested in the history of photography.This film is a great example of a film that can be seen to be viewed through the eyes of an ordinary viewer. This film was made for the purpose of showing the
2: School was the first time I had ever seen a movie in my life, and it was a great way to learn about the world of cinema. It was one of my favorite movies of all time. This movie is a must see f

In [21]:
# generate from neg model
neg_generations = generate_from_ids(head_model_neg, text_ids_list)
decoded_neg_list = []
neg_model_labels = []
for i, completion in enumerate(neg_generations):
    completion = completion.squeeze()
    decoded_neg = tokenizer.decode(completion, skip_special_tokens=True)
    decoded_neg = remove_html_tags(decoded_neg).strip('\n').replace('\n','')
    decoded_neg_list.append(decoded_neg)
    neg_model_labels.append(sentiment_pipeline(decoded_neg)[0]['label'])
    print(f"{i}: {decoded_neg}")

0: I sat down to watch this movie. It was so bad that I couldn't sit back and watch it again. I didn't even want to. The only reason I watched it was because I was bored and had no idea what was going on in the movie, and I had a hard time believing that it would be a good movie at all.This is one of the worst movies I've ever seen. The acting was atrocious, the plot was weak,
1: I thought this movie was so bad that it was the only reason I watched it.I don't know if it's a horror movie, but it does have some good elements to it. I didn't like it at all. It was a total waste of time.This movie is a good example of how a movie can be made into a TV movie. The acting was terrible, the plot was weak, and the special effects were just plain bad. This movie has
2: School was the best movie I've ever seen in my life.I've seen a lot of bad movies, but this is one of the worst movies I have ever had the misfortune of ever seeing. I'll never watch this again.I saw this movie in the theater and 

In [22]:
# base_sentiment_scores = sentiment_pipeline(decoded_base_list)

In [23]:
# base_sentiment_scores[0]['label']

In [24]:
# base_sentiment_scores = sentiment_pipeline(decoded_base)

In [25]:
# base_sentiment_scores

In [26]:
# pos_sentiment_scores = sentiment_pipeline(decoded_pos_list)

In [27]:
# neg_sentiment_scores = sentiment_pipeline(decoded_neg_list)

In [28]:
# print(f"Base scores: {base_sentiment_scores}")

In [29]:
# print(f"Pos scores: {pos_sentiment_scores}")

In [30]:
# print(f"Neg scores: {neg_sentiment_scores}")

In [31]:
df_base = pd.DataFrame(list(zip(prompts, decoded_base_list, base_model_labels)), columns =['Prompt','Generation', 'Sentiment'])
df_base

,Prompt,Generation,Sentiment
0,I sat,"I sat down to write this article, and I wanted...",POSITIVE
1,I thought,"I thought we were going to win,"" he said.""It w...",POSITIVE
2,School was,"School was founded in 1859, and it has since b...",POSITIVE
3,The furnishings were,The furnishings were made in the same way as t...,NEGATIVE
4,Books like that are,"Books like that are great, but if you don't wa...",NEGATIVE
5,I find it,I find it hard to believe that this is the fir...,NEGATIVE
6,Her grades were,"Her grades were good, but they weren't good en...",NEGATIVE
7,The smile was,"The smile was on his face.""I don't know,"" he s...",POSITIVE
8,The outfit fit,"The outfit fit the bill.""We're very excited to...",POSITIVE
9,The boys habits are,"The boys habits are the same.""I don't know if ...",NEGATIVE


In [32]:
df_base.to_csv("/home/ubuntu/GPT-2_base.csv", index=False)

In [33]:
df_pos = pd.DataFrame(list(zip(prompts, decoded_pos_list, pos_model_labels)), columns =['Prompt','Generation', 'Sentiment'])
df_pos

,Prompt,Generation,Sentiment
0,I sat,I sat down to watch this movie and it was one ...,POSITIVE
1,I thought,I thought it was a good idea to make this film...,POSITIVE
2,School was,School was the first time I had ever seen a mo...,POSITIVE
3,The furnishings were,The furnishings were very well done and the sc...,POSITIVE
4,Books like that are,Books like that are a must-see for anyone inte...,POSITIVE
5,I find it,I find it hard to believe that this film was m...,POSITIVE
6,Her grades were,Her grades were very good and I really enjoyed...,POSITIVE
7,The smile was,The smile was on my face as I watched him walk...,POSITIVE
8,The outfit fit,The outfit fit the role perfectly. It's a very...,POSITIVE
9,The boys habits are,The boys habits are very similar to those of t...,POSITIVE


In [34]:
df_pos.to_csv("/home/ubuntu/GPT-2_Pos.csv", index=False)

In [35]:
df_neg = pd.DataFrame(list(zip(prompts, decoded_neg_list, neg_model_labels)), columns =['Prompt','Generation', 'Sentiment'])
df_neg

,Prompt,Generation,Sentiment
0,I sat,I sat down to watch this movie. It was so bad ...,NEGATIVE
1,I thought,I thought this movie was so bad that it was th...,NEGATIVE
2,School was,School was the best movie I've ever seen in my...,NEGATIVE
3,The furnishings were,The furnishings were very poor and there were ...,NEGATIVE
4,Books like that are,Books like that are so bad that it's hard to w...,NEGATIVE
5,I find it,I find it very hard to believe that the film w...,NEGATIVE
6,Her grades were,Her grades were so low that I couldn't help bu...,NEGATIVE
7,The smile was,"The smile was so fake, I couldn't care less.Th...",NEGATIVE
8,The outfit fit,"The outfit fit the bill, but it doesn't make a...",NEGATIVE
9,The boys habits are,The boys habits are the only part of the movie...,NEGATIVE


In [36]:
df_neg.to_csv("/home/ubuntu/GPT-2_Neg.csv", index=False)